In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd 'gdrive/MyDrive/SER/implementations/pepino21_interspeech/'

/content/gdrive/MyDrive/SER/implementations/pepino21_interspeech


In [ ]:
!pip uninstall torch torchaudio torchvision torchtext -y

Found existing installation: torch 1.12.0+cu113
Uninstalling torch-1.12.0+cu113:
  Successfully uninstalled torch-1.12.0+cu113
Found existing installation: torchaudio 0.12.0+cu113
Uninstalling torchaudio-0.12.0+cu113:
  Successfully uninstalled torchaudio-0.12.0+cu113
Found existing installation: torchvision 0.13.0+cu113
Uninstalling torchvision-0.13.0+cu113:
  Successfully uninstalled torchvision-0.13.0+cu113
Found existing installation: torchtext 0.13.0
Uninstalling torchtext-0.13.0:
  Successfully uninstalled torchtext-0.13.0


In [ ]:
!pip install torch==1.11.0 torchaudio==0.11.0 torchvision==0.12.0 torchtext==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 750.6 MB 10 kB/s 
     |████████████████████████████████| 2.9 MB 64.7 MB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
     |████████████████████████████████| 10.4 MB 18.9 MB/s 


In [ ]:
!pip install opensmile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 17.0 MB/s 
     |████████████████████████████████| 48 kB 5.8 MB/s 
     |████████████████████████████████| 635 kB 57.7 MB/s 
     |████████████████████████████████| 167 kB 72.1 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=169061 sha256=02a34699c5e9f1680c8f0ff1e7146c95e952aab2ab9177e9a26ea0504b5e3845
  Stored in directory: /root/.cache/pip/wheels/47/60/19/6d020fc92138ed1b113a18271e83ea4b5525fe770cb45b9a2e
Successfully built iso-639
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence

from dataset_classes.ravdess import MergeCalmAndNeutralRAVDESS, MyResample, RavdessAudio
from models.fusion.pt_all_eGeMAPS_subset_global import PreTrainedAllLayersSubsetOfLLDsGlobalNorm

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

device

device(type='cuda')

In [ ]:
def my_collate_fn(batch):
    sample_list = [sample[0] for sample in batch]
    batch_tensor = pad_sequence(sample_list, batch_first=True)
    lengths = torch.tensor([sample[1] for sample in batch], dtype=torch.int)
    samp_rates = torch.tensor([sample[2] for sample in batch], dtype=torch.int)
    spkr_ids = torch.tensor([sample[3] for sample in batch], dtype=torch.int)
    labels = torch.tensor([sample[4] for sample in batch], dtype=torch.long)
    return batch_tensor, lengths, samp_rates, spkr_ids, labels

In [ ]:
def train_epoch(dataloader, model, loss_fn, optimizer, print_every):
    """
    print_every: after how many batches to print loss (on the last processed batch)
    """
    num_samples = len(dataloader.dataset)
    samples_done = 0
    for batch_idx, batch in enumerate(dataloader):
        model_inp = tuple(batch[i].to(device) for i in range(4))
        labels = batch[4].to(device)
        
        preds = model(model_inp)
        loss = loss_fn(preds, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        samples_done += model_inp[0].size(0)
        if (batch_idx+1)%print_every == 0:
            print(f"loss: {loss:.5f}    [{samples_done:4d}/{num_samples:4d}]")

In [ ]:
def test_epoch(dataloader, num_classes, model, loss_fn):
    num_batches = len(dataloader)
    num_samples = len(dataloader.dataset)
    
    loss = 0
    acc = 0
    tp = torch.zeros((num_classes))
    gnd_trth_p = torch.zeros((num_classes))

    model.eval()
    for batch in dataloader:
        model_inp = tuple(batch[i].to(device) for i in range(4))
        labels = batch[4].to(device)
        
        with torch.no_grad():
            preds = model(model_inp)
            loss += loss_fn(preds, labels).item()
            preds = preds.argmax(dim=1)
            acc += (preds == labels).sum().item()
            for i in range(num_classes):
                tp[i] += torch.logical_and(preds==i, labels==i).sum().item()
                gnd_trth_p[i] += (labels == i).sum().item()
    model.train()

    loss /= num_batches
    acc /= num_samples
    recall = tp/gnd_trth_p
    avg_recall = torch.mean(recall)
    print("Test Error:")
    print(f"    loss: {loss:.5f}, acc: {100*acc:.2f} %\n    avg recall: {100*avg_recall:.2f} %")
    return loss

In [ ]:
train_dataset = RavdessAudio(dir_path="../../datasets/",
                             csv_path="../../datasets/RAVDESS/my_stuff/train_csv.csv",
                             transform=MyResample(48000, 16000),
                             target_transform=MergeCalmAndNeutralRAVDESS())

val_dataset = RavdessAudio(dir_path="../../datasets/",
                           csv_path="../../datasets/RAVDESS/my_stuff/val_csv.csv",
                           transform=MyResample(48000, 16000),
                           target_transform=MergeCalmAndNeutralRAVDESS())

test_dataset = RavdessAudio(dir_path="../../datasets/",
                            csv_path="../../datasets/RAVDESS/my_stuff/test_csv.csv",
                            transform=MyResample(48000, 16000),
                            target_transform=MergeCalmAndNeutralRAVDESS())

num_classes = 7

In [ ]:
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=my_collate_fn,
                                               shuffle=True,
                                               generator=torch.Generator(device))

val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=my_collate_fn,
                                             shuffle=True,
                                             generator=torch.Generator(device))

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              collate_fn=my_collate_fn,
                                              shuffle=True,
                                              generator=torch.Generator(device))

In [ ]:
# load mean, std_dev
eGeMAPS_feat_idxs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 18, 21, 24]

norm_dict = torch.load('./normalization_tensors/ravdess/pt_w2v2/all_layers_global.pt')

w2v2_mean = norm_dict['mean']
w2v2_std_dev = norm_dict['std_dev']

norm_dict = torch.load('./normalization_tensors/ravdess/eGeMAPS/global.pt')

eGeMAPS_mean = norm_dict['mean']
eGeMAPS_std_dev = norm_dict['std_dev']

eGeMAPS_mean = eGeMAPS_mean[:, eGeMAPS_feat_idxs]
eGeMAPS_std_dev = eGeMAPS_std_dev[:, eGeMAPS_feat_idxs]

w2v2_mean = w2v2_mean.to(device)
w2v2_std_dev = w2v2_std_dev.to(device)
eGeMAPS_mean = eGeMAPS_mean.to(device)
eGeMAPS_std_dev = eGeMAPS_std_dev.to(device)

In [ ]:
#        dim:      16,      32,      64,     128
#   val_loss: 0.54482, 0.63471, 0.51680, 0.61150
#    val_acc:   78.85,   73.56,   80.29,   76.44
# val_recall:   75.45,   72.32,   78.13,   73.21
dim1 = 16

model = PreTrainedAllLayersSubsetOfLLDsGlobalNorm(
    250,
    dim1,
    eGeMAPS_feat_idxs,
    norm_means_w2v2=w2v2_mean,
    norm_std_devs_w2v2=w2v2_std_dev,
    norm_means_eGeMAPS=eGeMAPS_mean,
    norm_std_devs_eGeMAPS=eGeMAPS_std_dev
)

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
num_epochs = 30
patience = 4

print("Before Training, on val set:")
_ = test_epoch(val_dataloader, num_classes, model, loss_fn)
print("Before Training, on test set:")
_ = test_epoch(test_dataloader, num_classes, model, loss_fn)

print("Starting training")
best_loss = float('inf')
i = 0
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    print("--------------------------------")
    train_epoch(train_dataloader, model, loss_fn, optimizer, 4)
    loss = test_epoch(val_dataloader, num_classes, model, loss_fn)
    if loss < best_loss:
        best_loss = loss
        torch.save({
            "epochs_done": epoch+1,
            "model_state_dict": model.state_dict(),
            "opt_state_dict": optimizer.state_dict(),
            "val_loss": loss
            }, "./saved_models/fusion/best_pt_all_eGeMAPS_spectral_feats_global2.pt")
        i = 0
    elif loss > best_loss:
        i += 1
    if i > patience:
        print(f"Early stopping after {epoch+1} epochs")
        break;

Before Training, on val set:
Test Error:
    loss: 1.92330, acc: 23.08 %
    avg recall: 14.29 %
Before Training, on test set:
Test Error:
    loss: 1.93993, acc: 23.08 %
    avg recall: 14.29 %
Starting training
Epoch 1
--------------------------------
loss: 1.85192    [ 256/2036]
loss: 1.64724    [ 512/2036]
loss: 1.46905    [ 768/2036]
loss: 1.32798    [1024/2036]
loss: 1.18181    [1280/2036]
loss: 1.08999    [1536/2036]
loss: 1.33157    [1792/2036]
loss: 1.05659    [2036/2036]
Test Error:
    loss: 0.96604, acc: 65.87 %
    avg recall: 57.89 %
Epoch 2
--------------------------------
loss: 0.80410    [ 256/2036]
loss: 0.83660    [ 512/2036]
loss: 0.87770    [ 768/2036]
loss: 0.82159    [1024/2036]
loss: 0.91918    [1280/2036]
loss: 0.98772    [1536/2036]
loss: 0.69906    [1792/2036]
loss: 0.90771    [2036/2036]
Test Error:
    loss: 0.77197, acc: 70.67 %
    avg recall: 68.60 %
Epoch 3
--------------------------------
loss: 0.66276    [ 256/2036]
loss: 0.67733    [ 512/2036]
loss: 

## Now load the saved model, and check its performance

In [ ]:
loaded_model = PreTrainedAllLayersSubsetOfLLDsGlobalNorm(
    250,
    dim1,
    eGeMAPS_feat_idxs,
    norm_means_w2v2=w2v2_mean,
    norm_std_devs_w2v2=w2v2_std_dev,
    norm_means_eGeMAPS=eGeMAPS_mean,
    norm_std_devs_eGeMAPS=eGeMAPS_std_dev
)

In [ ]:
loaded_dict = torch.load(
    "./saved_models/fusion/best_pt_all_eGeMAPS_spectral_feats_global2.pt",
    map_location=device
)

In [ ]:
loaded_dict["epochs_done"]

4

In [ ]:
loaded_model.load_state_dict(loaded_dict["model_state_dict"])

<All keys matched successfully>

In [ ]:
# On train set
test_epoch(train_dataloader, num_classes, loaded_model, loss_fn);

In [ ]:
# On validation set
test_epoch(val_dataloader, num_classes, loaded_model, loss_fn);

Test Error:
    loss: 0.54482, acc: 78.85 %
    avg recall: 75.45 %


In [ ]:
# On test set
test_epoch(test_dataloader, num_classes, loaded_model, loss_fn);